__Subida de datos a [doccano](https://doccano.github.io/doccano/)__

__Paso 1:__
* Instalar la libreria: ```!pip install doccano```
* A continuación, iniciar el servidor:

```
# Initialize database. First time only.
doccano init
# Create a super user. First time only.
doccano createuser --username admin --password pass
# Start a web server.
doccano webserver --port 8000
```

* En otro terminal, ejecutar el siguiente comando:

```
doccano task
```

De cara a subir el dataset, seguir [los siguientes pasos](https://doccano.github.io/doccano/tutorial/).

In [15]:
# -- Convertimos la salida del modelo de OpenAI en el formato de doccano
import pandas as pd
import re
import os
output_0 = pd.read_json("/Users/alberto/Personal Projects/hackathon_podcast/data/NER/json_entity/json_files_alberto/output_0.json")
output_0['fe_entities'] = output_0.apply(lambda x: [(x_aux['class'], x_aux['text'].lower(), re.search(x_aux['text'].lower(), x['text'].lower())) for x_aux in x['entities']\
                                                    if x_aux['text'] != ""], axis=1)
output_0['fe_entities_start_end'] = output_0['fe_entities'].apply(lambda x: [[x_aux[-1].start(), x_aux[-1].end(), x_aux[0].upper()] for x_aux in x if x_aux[-1] != None])

__Doccano format__:

```
{"text": "EU rejects German call to boycott British lamb.", "label": [[0, 2, "ORG"]]}
{"text": "Peter Blackburn", "label": [[0, 15, "PERSON"]]}
{"text": "President Obama", "label": [[10, 15, "PERSON"]]}
```

In [16]:
output_0['fe_final_dict'] = output_0.apply(lambda x: {"text": x["text"], "label": x["fe_entities_start_end"]}, axis=1)
output_0

,text,entities,fe_entities,fe_entities_start_end,fe_final_dict
0,"A ver, voy a decir yo las tres mejores históri...","[{'class': 'people names', 'text': 'yo'}, {'cl...","[(people names, yo, <re.Match object; span=(19...","[[19, 21, PEOPLE NAMES], [22, 49, BOOK NAMES],...","{'text': 'A ver, voy a decir yo las tres mejor..."
1,Pero te encanta en rollo que te lo pasas bien ...,"[{'class': 'people names', 'text': ''}, {'clas...",[],[],{'text': 'Pero te encanta en rollo que te lo p...
2,Creo que la estructura de Guillaume con varios...,"[{'class': 'film names', 'text': 'Rosebud'}, {...","[(film names, rosebud, <re.Match object; span=...","[[212, 219, FILM NAMES], [26, 35, PEOPLE NAMES]]",{'text': 'Creo que la estructura de Guillaume ...
3,"muy, muy interesante, y a nivel de imagen y ta...","[{'class': 'places', 'text': 'nivel de imagen'...","[(places, nivel de imagen, <re.Match object; s...","[[26, 41, PLACES], [0, 3, ADJECTIVES], [0, 3, ...","{'text': 'muy, muy interesante, y a nivel de i..."
4,"O sea, no me aburre, no me gusta mucho.\n","[{'class': 'None', 'text': 'O sea'}, {'class':...","[(None, o sea, <re.Match object; span=(0, 5), ...","[[0, 5, NONE], [7, 19, NONE], [21, 38, NONE]]","{'text': 'O sea, no me aburre, no me gusta muc..."
...,...,...,...,...,...
115,Es tan… porque Jack The Ripper tiene algunas p...,"[{'class': 'people names', 'text': 'Jack The R...","[(people names, jack the ripper, <re.Match obj...","[[15, 30, PEOPLE NAMES], [59, 76, FILM NAMES]]",{'text': 'Es tan… porque Jack The Ripper tiene...
116,Donnie Darko es una obra maestra.\n,"[{'class': 'film names', 'text': 'Donnie Darko...","[(film names, donnie darko, <re.Match object; ...","[[0, 12, FILM NAMES], [20, 32, BOOK NAMES]]","{'text': 'Donnie Darko es una obra maestra. ',..."
117,"Bueno, es una obra maestra, pero también tiene...","[{'class': 'book names', 'text': 'obra maestra...","[(book names, obra maestra, <re.Match object; ...","[[14, 26, BOOK NAMES], [89, 101, PLACES]]","{'text': 'Bueno, es una obra maestra, pero tam..."
118,"Me he dicho mis tres favoritas, ¿no?\n","[{'class': 'people names', 'text': 'Me'}, {'cl...","[(people names, me, <re.Match object; span=(0,...","[[0, 2, PEOPLE NAMES], [6, 11, PEOPLE NAMES]]","{'text': 'Me he dicho mis tres favoritas, ¿no?..."


__Por último, exportamos el diccionario a .jsonl__

In [18]:
# Abre un archivo de salida en modo de escritura
import json
with open('output.jsonl', 'w') as f:
    # Itera a través de las filas del DataFrame
    for _, row in output_0.iterrows():
        # Convierte el diccionario en una cadena JSON
        json_str = json.dumps(row['fe_final_dict'])
        # Escribe la cadena JSON en el archivo de salida, seguida de un salto de línea
        f.write(json_str + '\n')